### Summarizing Long Articles

In [33]:
from dotenv import load_dotenv
import os
from langchain.chat_models import AzureChatOpenAI
#import textwrap
from IPython.display import display, HTML, JSON, Markdown

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION")

client = AzureChatOpenAI(
  azure_endpoint = OPENAI_DEPLOYMENT_ENDPOINT, 
  api_key=OPENAI_API_KEY,  
  api_version="2023-05-15"
)

In [39]:
from langchain import PromptTemplate
from langchain import text_splitter
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader

fileName = "./data/Best-Practices-in-Change-Management-Full-Report-Digital-11th-Edition.pdf"
loader = PyPDFLoader(fileName)
pages = loader.load_and_split()
print("Number of pages: ", len(pages))

Number of pages:  367


In [41]:
def init_llm(model=OPENAI_MODEL_NAME,
             deployment_name=OPENAI_DEPLOYMENT_NAME,
             openai_api_version=OPENAI_DEPLOYMENT_VERSION,
             azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
             temperature=0,
             max_tokens=3000,
             top_p=1,
             ):

    llm = AzureChatOpenAI(deployment_name=deployment_name,
                      model=model,
                      openai_api_version=openai_api_version,
                      azure_endpoint=azure_endpoint,
                      temperature=temperature,
                      max_tokens=max_tokens,
                      top_p=top_p,
                      model_kwargs={"stop": ["<|im_end|>"]}
                      )
    return llm
llm = init_llm()

WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.


In [43]:
map_prompt_template = """
                      Write a summary of this chunk of text that includes the main points and any important details.
                      {text}
                      """

map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])

combine_prompt_template = """
                      Write a concise summary of the following text delimited by triple backquotes.
                      Return your response in bullet points which covers the key points of the text.
                      ```{text}```
                      BULLET POINT SUMMARY:
                      """

combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["text"]
)

_load_summarize_chain_ has 3 types of chains:
1. **stuff** - The chain will take a list of documents, inserts them all into a prompt, and passes that prompt to the LLM.
2. **map_reduce** - First, map each chunk to an individual summary using an LLMChain. Then, combine those summaries into a single global summary.
3. **refine** -  Loop over the chunks and iteratively update the answer. For each chunk, it passes the meta-prompt, the current document, and the latest intermediate summary to an LLM chain to get a new summary.

The following example is demonstrates the "map reduce" chain type. 

In [44]:
map_reduce_chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=combine_prompt,
    return_intermediate_steps=True,
)

In [45]:
map_reduce_outputs = map_reduce_chain({"input_documents": pages})
print("Number of outputs: ", len(map_reduce_outputs))

In order to display the content of the _map_reduce_outputs_ object, we will use _pandas_ library which will help us to display the output as a table.

In [24]:
import pandas as pd
from pathlib import Path as p
final_mp_data = []
for doc, out in zip(map_reduce_outputs["input_documents"], map_reduce_outputs["intermediate_steps"]):
    output = {}
    output["file_name"] = p(doc.metadata["source"]).stem
    output["file_type"] = p(doc.metadata["source"]).suffix
    output["page_number"] = doc.metadata["page"]
    output["chunks"] = doc.page_content
    output["concise_summary"] = out
    final_mp_data.append(output)

pdf_mp_summary = pd.DataFrame.from_dict(final_mp_data)
pdf_mp_summary = pdf_mp_summary.sort_values(
    by=["file_name", "page_number"]
)  # sorting the dataframe by filename and page_number
pdf_mp_summary.reset_index(inplace=True, drop=True)
pdf_mp_summary.head()

,file_name,file_type,page_number,chunks,concise_summary
0,Best-Practices-in-Change-Management-Full-Repor...,.pdf,0,11th EditionBest Practices In \nChange Manage...,The 11th edition of the Prosci Benchmarking Re...
1,Best-Practices-in-Change-Management-Full-Repor...,.pdf,1,© 2020 Prosci Inc. • 1\nProsci ® Best Pra...,The text is a summary of the 11th edition of B...
2,Best-Practices-in-Change-Management-Full-Repor...,.pdf,3,© 2020 Prosci Inc. • 3\nProsci ® Best Pra...,This text is a table of contents for a documen...
3,Best-Practices-in-Change-Management-Full-Repor...,.pdf,3,Chapter 6 : Methodology ........................,"In Chapter 6, the methodology for initiating c..."
4,Best-Practices-in-Change-Management-Full-Repor...,.pdf,4,4 • © 2020 Prosci Inc. \nChapter 8 : Change ...,This text is a table of contents for a book on...


In [23]:
index = 3
print("[Context]")
print(pdf_mp_summary["chunks"].iloc[index])
print("\n\n [Simple Summary]")
print(pdf_mp_summary["concise_summary"].iloc[index])
print("\n\n [Page number]")
print(pdf_mp_summary["page_number"].iloc[index])
print("\n\n [Source: file_name]")
print(pdf_mp_summary["file_name"].iloc[index])

[Context]
Chapter 6 : Methodology  ............................................................................................................................. 105
Initiating Change Management .............................................................................................................. 107
Application of Methodology  ................................................................................................................... 110
Methodologies Used  ............................................................................................................................... 114
Chapter 7 : CM Budget, Resources and Team Structure  ...................................................................... 119
Change Management Budget  ................................................................................................................ 121
Change Management Resources  ...........................................................................................